![title](imgs/cover.png)

# Deploy dos modelos de Machine Learning
A quarta etapa foca no deploy e na aplicação dos modelos de machine learning. Inicia-se com a importação dos modelos, artefatos e dados necessários para realizar previsões. Com todos os componentes em mãos, os dados são preparados seguindo o mesmo procedimento utilizado no treinamento dos modelos, garantindo que as previsões sejam feitas com a máxima precisão. Isso envolve aplicar encoders nas variáveis categóricas e normalizar as variáveis numéricas. Uma vez que os dados estão preparados, cada um dos três modelos selecionados é usado para fazer previsões. Esta etapa é vital para testar a eficácia dos modelos em condições reais e observar como cada um responde a novos dados. O processo completo não só permite a aplicação prática dos modelos de machine learning, mas também estabelece a base para futuras iterações e melhorias baseadas nos resultados obtidos.

---

In [1]:
import pandas as pd

from joblib import load

---
# Importação dos Artefatos, Modelos e Dados
Esta seção é dedicada à importação de todos os recursos necessários para o deployment dos modelos. Isso inclui carregar os modelos de machine learning previamente salvos, bem como os artefatos como encoders e scalers, e os dados sobre os quais as previsões serão feitas.

In [2]:
# Carregando data preparation
encoder_loaded = load('../models/ordinal_encoder.joblib')
scaler_loaded = load('../models/scaler.joblib')

# Carregar as variáveis selecionadas
loaded_features = load('../models/selected_features.pkl')

# Carregar o modelos
best_rf_model = load('../models/best_rf_model.pkl')
best_ada_model = load('../models/best_ada_model.pkl')
best_svm_model = load('../models/best_svm_model.pkl')

In [3]:
# Import data
df = pd.read_csv('data/cleaned_test_data.csv', sep = ';')

In [4]:
print(df.shape)
df.head()

(585340, 14)


,country,creation_platform,admin_visits,intercom_conversations,source_pulido,creation_weekday,creation_hour,products_with_description,total_products_with_images,total_product_categories,total_events_on_Android,total_events_on_Web,total_events_on_iOS,target
0,BR,mobile_app,1.0,0.0,Other,6,3,2.0,1,0.0,215.0,6.0,0.0,0
1,BR,mobile_web,2.0,0.0,Google CPC no Brand,6,3,1.0,1,0.0,0.0,44.0,0.0,0
2,BR,mobile_web,1.0,0.0,Google CPC no Brand,6,3,0.0,0,0.0,0.0,16.0,0.0,0
3,BR,mobile_web,1.0,0.0,Google CPC no Brand,6,3,0.0,0,0.0,0.0,15.0,0.0,0
4,BR,mobile_web,1.0,0.0,Google CPC no Brand,6,3,0.0,0,0.0,0.0,24.0,0.0,0


---
# Preparação dos Dados com os Artefatos Importados
Após a importação, os dados precisam ser preparados usando os mesmos artefatos aplicados durante o treinamento dos modelos. Esta etapa assegura que os dados estejam na forma correta para serem utilizados pelos modelos treinados. A preparação inclui a aplicação dos encoders nas variáveis categóricas e a normalização das variáveis numéricas e a seleção das variáveis utilizadas para treinar os modelos

In [5]:
# Selecionar uma amostra aleatória do DataFrame original
df_random_sample = df.sample(1)

In [6]:
# Checando quais colunas são do tipo 'object', indicando variáveis categóricas
categorical_columns = df_random_sample.select_dtypes(include=['object']).columns

# Identificando colunas numéricas (int e float)
numeric_columns = df_random_sample.select_dtypes(include=['int64', 'float64']).drop(columns='target').columns

In [7]:
df_random_sample[categorical_columns] = encoder_loaded.transform(df_random_sample[categorical_columns])
df_random_sample[numeric_columns] = scaler_loaded.transform(df_random_sample[numeric_columns])

df_random_sample_filter = df_random_sample[loaded_features]

df_random_sample.head()

,country,creation_platform,admin_visits,intercom_conversations,source_pulido,creation_weekday,creation_hour,products_with_description,total_products_with_images,total_product_categories,total_events_on_Android,total_events_on_Web,total_events_on_iOS,target
360483,1.0,1.0,-0.162889,-0.071664,7.0,-0.888361,0.07843,-0.053906,-0.077705,-0.188791,0.595344,-0.578073,-0.073609,0


---
# Realização de Previsões com os Modelos
Com os dados devidamente preparados, esta etapa foca em utilizar cada um dos três modelos selecionados para realizar previsões.

In [8]:
# ----- Reinforcement Learning
# Previsão
rf_prediction = best_rf_model.predict(df_random_sample_filter)
rf_probabilities = best_rf_model.predict_proba(df_random_sample_filter)

# Exibir a predição e as probabilidades para cada classe
print("\n----- Reinforcement Learning -----\n")
print(f"Prediction: {rf_prediction}")

# Se desejar apresentar as probabilidades como porcentagens:
rf_percentages = rf_probabilities * 100
print(f"Percentage for Class 0: {rf_percentages[0][0]:.2f}%")
print(f"Percentage for Class 1: {rf_percentages[0][1]:.2f}%")

# ----- AdaBoost
# Previsão 
ada_prediction = best_ada_model.predict(df_random_sample_filter)
ada_probabilities = best_ada_model.predict_proba(df_random_sample_filter)

# Exibir a predição e as probabilidades para cada classe
print("\n----- AdaBoost -----\n")
print(f"Prediction: {ada_prediction}")

# Se desejar apresentar as probabilidades como porcentagens:
ada_percentages = ada_probabilities * 100
print(f"Percentage for Class 0: {ada_percentages[0][0]:.2f}%")
print(f"Percentage for Class 1: {ada_percentages[0][1]:.2f}%")

# ----- SVM
# Previsão 
svm_prediction = best_svm_model.predict(df_random_sample_filter)
svm_probabilities = best_svm_model.predict_proba(df_random_sample_filter)

# Exibir a predição e as probabilidades para cada classe
print("\n----- SVM -----\n")
print(f"Prediction: {svm_prediction}")

# Se desejar apresentar as probabilidades como porcentagens:
svm_percentages = svm_probabilities * 100
print(f"Percentage for Class 0: {svm_percentages[0][0]:.2f}%")
print(f"Percentage for Class 1: {svm_percentages[0][1]:.2f}%")


----- Reinforcement Learning -----

Prediction: [0]
Percentage for Class 0: 97.98%
Percentage for Class 1: 2.02%

----- AdaBoost -----

Prediction: [0]
Percentage for Class 0: 61.04%
Percentage for Class 1: 38.96%

----- SVM -----

Prediction: [0]
Percentage for Class 0: 92.25%
Percentage for Class 1: 7.75%


![title](imgs/end.png)